In [1]:
pwd


'/Volumes/data/Kaggle/marchmadness'

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [3]:
teams = pd.read_csv("./Teams.csv", index_col=0)

In [4]:
teams.head()

,Team_Name
Team_Id,
1101,Abilene Chr
1102,Air Force
1103,Akron
1104,Alabama
1105,Alabama A&M


In [5]:
print(teams[teams['Team_Name']=='Georgia Tech'])

            Team_Name
Team_Id              
1210     Georgia Tech


In [6]:
rscr = pd.read_csv("./RegularSeasonCompactResults.csv")

In [7]:
rsdr = pd.read_csv("./RegularSeasonDetailedResults.csv")

In [8]:
tcr = pd.read_csv("./TourneyCompactResults.csv")

In [9]:
ss = pd.read_csv("./sample_submission.csv")

In [10]:
rscr.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [11]:
rscr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
145284,2016,132,1114,70,1419,50,N,0
145285,2016,132,1163,72,1272,58,N,0
145286,2016,132,1246,82,1401,77,N,1
145287,2016,132,1277,66,1345,62,N,0
145288,2016,132,1386,87,1433,74,N,0


In [12]:
rsdr.head().transpose()

,0,1,2,3,4
Season,2003,2003,2003,2003,2003
Daynum,10,10,11,11,11
Wteam,1104,1272,1266,1296,1400
Wscore,68,70,73,56,77
Lteam,1328,1393,1437,1457,1208
Lscore,62,63,61,50,71
Wloc,N,N,N,N,N
Numot,0,0,0,0,0
Wfgm,27,26,24,18,30
Wfga,58,62,58,38,61


In [13]:
rsdr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
71236,2016,132,1114,70,1419,50,N,0,26,52,...,18,4,9,6,20,13,13,6,3,12
71237,2016,132,1163,72,1272,58,N,0,22,48,...,16,11,17,5,21,10,6,3,0,20
71238,2016,132,1246,82,1401,77,N,1,28,58,...,23,15,22,17,23,11,13,5,4,20
71239,2016,132,1277,66,1345,62,N,0,25,60,...,15,17,21,5,22,10,5,4,4,14
71240,2016,132,1386,87,1433,74,N,0,35,54,...,29,11,16,12,21,12,9,5,5,21


In [14]:
top_10_reg = teams.join(pd.DataFrame(rscr['Wteam'].value_counts()[:9]), how='right')
top_10_reg.columns = ['Team_Name', 'Total_Wins']
top_10_reg

,Team_Name,Total_Wins
1181,Duke,819
1242,Kansas,804
1246,Kentucky,765
1314,North Carolina,761
1112,Arizona,746
1393,Syracuse,738
1272,Memphis,692
1424,UNLV,685
1257,Louisville,682


In [15]:
ss.head()

,id,pred
0,2013_1103_1107,0.5
1,2013_1103_1112,0.5
2,2013_1103_1125,0.5
3,2013_1103_1129,0.5
4,2013_1103_1137,0.5


The feature we are trying to predict is the probability that team1 will win. In order to do this effectively we need to choose or engineer numerical features that we'll know prior to the game occurring. The makeup of each team varies greatly from year to year. With that in mind I've decided to isolate my train and test data by year. 

In [16]:
rs2013 = rsdr[rsdr['Season']==2013]
rs2014 = rsdr[rsdr['Season']==2014]
rs2015 = rsdr[rsdr['Season']==2015]
rs2016 = rsdr[rsdr['Season']==2016]

In [17]:
rs2013_d1 = pd.DataFrame()
rs2014_d1 = pd.DataFrame()
rs2015_d1 = pd.DataFrame()
rs2016_d1 = pd.DataFrame()

In [18]:
rs2013_d2 = pd.DataFrame()
rs2014_d2 = pd.DataFrame()
rs2015_d2 = pd.DataFrame()
rs2016_d2 = pd.DataFrame()

In [19]:
rs2013_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2013[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2014_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2014[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2015_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2015[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2016_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2016[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()

In [20]:
rs2013_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2013[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2014_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2014[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2015_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2015[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2016_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2016[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()

In [21]:
rs2013_d1['pred'] = 1
rs2014_d1['pred'] = 1
rs2015_d1['pred'] = 1
rs2016_d1['pred'] = 1

In [22]:
rs2013_d2['pred'] = 0
rs2014_d2['pred'] = 0
rs2015_d2['pred'] = 0
rs2016_d2['pred'] = 0

In [23]:
rs2013_train = pd.concat([rs2013_d1, rs2013_d2]).reset_index(drop=True)
rs2014_train = pd.concat([rs2014_d1, rs2014_d2]).reset_index(drop=True)
rs2015_train = pd.concat([rs2015_d1, rs2015_d2]).reset_index(drop=True)
rs2016_train = pd.concat([rs2016_d1, rs2016_d2]).reset_index(drop=True)

In [24]:
rs2013_train['t1_fg_pct'] = rs2013_train['t1_fgm'] / rs2013_train['t1_fga']
rs2013_train['t1_fg3_pct'] = rs2013_train['t1_fgm3'] / rs2013_train['t1_fga3']
rs2013_train['t1_ft_pct'] = rs2013_train['t1_ftm'] / rs2013_train['t1_fta']
rs2014_train['t1_fg_pct'] = rs2014_train['t1_fgm'] / rs2014_train['t1_fga']
rs2014_train['t1_fg3_pct'] = rs2014_train['t1_fgm3'] / rs2014_train['t1_fga3']
rs2014_train['t1_ft_pct'] = rs2014_train['t1_ftm'] / rs2014_train['t1_fta']
rs2015_train['t1_fg_pct'] = rs2015_train['t1_fgm'] / rs2015_train['t1_fga']
rs2015_train['t1_fg3_pct'] = rs2015_train['t1_fgm3'] / rs2015_train['t1_fga3']
rs2015_train['t1_ft_pct'] = rs2015_train['t1_ftm'] / rs2015_train['t1_fta']
rs2016_train['t1_fg_pct'] = rs2016_train['t1_fgm'] / rs2016_train['t1_fga']
rs2016_train['t1_fg3_pct'] = rs2016_train['t1_fgm3'] / rs2016_train['t1_fga3']
rs2016_train['t1_ft_pct'] = rs2016_train['t1_ftm'] / rs2016_train['t1_fta']

In [25]:
rs2013_train['t2_fg_pct'] = rs2013_train['t2_fgm'] / rs2013_train['t2_fga']
rs2013_train['t2_fg3_pct'] = rs2013_train['t2_fgm3'] / rs2013_train['t2_fga3']
rs2013_train['t2_ft_pct'] = rs2013_train['t2_ftm'] / rs2013_train['t2_fta']
rs2014_train['t2_fg_pct'] = rs2014_train['t2_fgm'] / rs2014_train['t2_fga']
rs2014_train['t2_fg3_pct'] = rs2014_train['t2_fgm3'] / rs2014_train['t2_fga3']
rs2014_train['t2_ft_pct'] = rs2014_train['t2_ftm'] / rs2014_train['t2_fta']
rs2015_train['t2_fg_pct'] = rs2015_train['t2_fgm'] / rs2015_train['t2_fga']
rs2015_train['t2_fg3_pct'] = rs2015_train['t2_fgm3'] / rs2015_train['t2_fga3']
rs2015_train['t2_ft_pct'] = rs2015_train['t2_ftm'] / rs2015_train['t2_fta']
rs2016_train['t2_fg_pct'] = rs2016_train['t2_fgm'] / rs2016_train['t2_fga']
rs2016_train['t2_fg3_pct'] = rs2016_train['t2_fgm3'] / rs2016_train['t2_fga3']
rs2016_train['t2_ft_pct'] = rs2016_train['t2_ftm'] / rs2016_train['t2_fta']

In [26]:
rs2013_train = rs2013_train.sort_values('Daynum')
rs2014_train = rs2013_train.sort_values('Daynum')
rs2015_train = rs2013_train.sort_values('Daynum')
rs2016_train = rs2013_train.sort_values('Daynum')

In [27]:
t1_fg_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_ast_2013 = rs2013_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_ast_avg'] = t1_ast_2013.fillna(0).reset_index(level=0, drop=True)

t1_dr_2013 = rs2013_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_dr_avg'] = t1_dr_2013.fillna(0).reset_index(level=0, drop=True)

t1_stl_2013 = rs2013_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_stl_avg'] = t1_stl_2013.fillna(0).reset_index(level=0, drop=True)

t1_pf_2013 = rs2013_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_pf_avg'] = t1_pf_2013.fillna(0).reset_index(level=0, drop=True)

t1_blk_2013 = rs2013_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_blk_avg'] = t1_blk_2013.fillna(0).reset_index(level=0, drop=True)

In [28]:
t1_fg_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_ast_2014 = rs2014_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_ast_avg'] = t1_ast_2014.fillna(0).reset_index(level=0, drop=True)

t1_dr_2014 = rs2014_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_dr_avg'] = t1_dr_2014.fillna(0).reset_index(level=0, drop=True)

t1_stl_2014 = rs2014_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_stl_avg'] = t1_stl_2014.fillna(0).reset_index(level=0, drop=True)

t1_pf_2014 = rs2014_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_pf_avg'] = t1_pf_2014.fillna(0).reset_index(level=0, drop=True)

t1_blk_2014 = rs2014_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_blk_avg'] = t1_blk_2014.fillna(0).reset_index(level=0, drop=True)

In [29]:
t1_fg_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_ast_2015 = rs2015_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_ast_avg'] = t1_ast_2015.fillna(0).reset_index(level=0, drop=True)

t1_dr_2015 = rs2015_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_dr_avg'] = t1_dr_2015.fillna(0).reset_index(level=0, drop=True)

t1_stl_2015 = rs2015_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_stl_avg'] = t1_stl_2015.fillna(0).reset_index(level=0, drop=True)

t1_pf_2015 = rs2015_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_pf_avg'] = t1_pf_2015.fillna(0).reset_index(level=0, drop=True)

t1_blk_2015 = rs2015_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_blk_avg'] = t1_blk_2015.fillna(0).reset_index(level=0, drop=True)

In [30]:
t1_fg_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_ast_2016 = rs2016_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_ast_avg'] = t1_ast_2016.fillna(0).reset_index(level=0, drop=True)

t1_dr_2016 = rs2016_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_dr_avg'] = t1_dr_2016.fillna(0).reset_index(level=0, drop=True)

t1_stl_2016 = rs2016_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_stl_avg'] = t1_stl_2016.fillna(0).reset_index(level=0, drop=True)

t1_pf_2016 = rs2016_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_pf_avg'] = t1_pf_2016.fillna(0).reset_index(level=0, drop=True)

t1_blk_2016 = rs2016_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_blk_avg'] = t1_blk_2016.fillna(0).reset_index(level=0, drop=True)

In [31]:
t2_fg_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_ast_2013 = rs2013_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_ast_avg'] = t2_ast_2013.fillna(0).reset_index(level=0, drop=True)

t2_dr_2013 = rs2013_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_dr_avg'] = t2_dr_2013.fillna(0).reset_index(level=0, drop=True)

t2_stl_2013 = rs2013_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_stl_avg'] = t2_stl_2013.fillna(0).reset_index(level=0, drop=True)

t2_pf_2013 = rs2013_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_pf_avg'] = t2_pf_2013.fillna(0).reset_index(level=0, drop=True)

t2_blk_2013 = rs2013_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_blk_avg'] = t2_blk_2013.fillna(0).reset_index(level=0, drop=True)



In [32]:
t2_fg_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_ast_2014 = rs2014_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_ast_avg'] = t2_ast_2014.fillna(0).reset_index(level=0, drop=True)

t2_dr_2014 = rs2014_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_dr_avg'] = t2_dr_2014.fillna(0).reset_index(level=0, drop=True)

t2_stl_2014 = rs2014_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_stl_avg'] = t2_stl_2014.fillna(0).reset_index(level=0, drop=True)

t2_pf_2014 = rs2014_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_pf_avg'] = t2_pf_2014.fillna(0).reset_index(level=0, drop=True)

t2_blk_2014 = rs2014_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_blk_avg'] = t2_blk_2014.fillna(0).reset_index(level=0, drop=True)



In [33]:
t2_fg_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_ast_2015 = rs2015_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_ast_avg'] = t2_ast_2015.fillna(0).reset_index(level=0, drop=True)

t2_dr_2015 = rs2015_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_dr_avg'] = t2_dr_2015.fillna(0).reset_index(level=0, drop=True)

t2_stl_2015 = rs2015_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_stl_avg'] = t2_stl_2015.fillna(0).reset_index(level=0, drop=True)

t2_pf_2015 = rs2015_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_pf_avg'] = t2_pf_2015.fillna(0).reset_index(level=0, drop=True)

t2_blk_2015 = rs2015_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_blk_avg'] = t2_blk_2015.fillna(0).reset_index(level=0, drop=True)



In [34]:
t2_fg_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_ast_2016 = rs2016_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_ast_avg'] = t2_ast_2016.fillna(0).reset_index(level=0, drop=True)

t2_dr_2016 = rs2016_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_dr_avg'] = t2_dr_2016.fillna(0).reset_index(level=0, drop=True)

t2_stl_2016 = rs2016_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_stl_avg'] = t2_stl_2016.fillna(0).reset_index(level=0, drop=True)

t2_pf_2016 = rs2016_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_pf_avg'] = t2_pf_2016.fillna(0).reset_index(level=0, drop=True)

t2_blk_2016 = rs2016_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_blk_avg'] = t2_blk_2016.fillna(0).reset_index(level=0, drop=True)



In [35]:
target13 = rs2013_train['pred'].values
target14 = rs2014_train['pred'].values
target15 = rs2015_train['pred'].values
target16 = rs2016_train['pred'].values

In [36]:
features_array13 = rs2013_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array14 = rs2014_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array15 = rs2015_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array16 = rs2016_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values

In [37]:
logreg = LogisticRegression(C=1)
lrscores13 = cross_val_score(logreg, features_array13, target13, cv=5)
lrscores14 = cross_val_score(logreg, features_array14, target14, cv=5)
lrscores15 = cross_val_score(logreg, features_array15, target15, cv=5)
lrscores16 = cross_val_score(logreg, features_array16, target16, cv=5)

In [38]:
lrscores13.mean()
lrscores14.mean()
lrscores15.mean()
lrscores16.mean()

0.62546992481203012

In [39]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rfscores13 = cross_val_score(rf, features_array13, target13, cv=5, n_jobs=4, scoring='accuracy')
rfscores14 = cross_val_score(rf, features_array14, target14, cv=5, n_jobs=4, scoring='accuracy')
rfscores15 = cross_val_score(rf, features_array15, target15, cv=5, n_jobs=4, scoring='accuracy')
rfscores16 = cross_val_score(rf, features_array16, target16, cv=5, n_jobs=4, scoring='accuracy')

print('Random Forest CV Scores:')
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores13.min(), rfscores13.mean(), rfscores13.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores14.min(), rfscores14.mean(), rfscores14.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores15.min(), rfscores15.mean(), rfscores15.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores16.min(), rfscores16.mean(), rfscores16.max()))

Random Forest CV Scores:
min: 0.602, mean; 0.640, max: 0.694
min: 0.591, mean; 0.640, max: 0.696
min: 0.591, mean; 0.640, max: 0.696
min: 0.591, mean; 0.640, max: 0.696


In [40]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, subsample=.8, max_features=.5)

gbscores13 = cross_val_score(gb, features_array13, target13, cv=10, n_jobs=4, scoring='accuracy')
gbscores14 = cross_val_score(gb, features_array14, target14, cv=10, n_jobs=4, scoring='accuracy')
gbscores15 = cross_val_score(gb, features_array15, target15, cv=10, n_jobs=4, scoring='accuracy')
gbscores16 = cross_val_score(gb, features_array16, target16, cv=10, n_jobs=4, scoring='accuracy')

print("Gradient Boosted Trees CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores13.min(), gbscores13.mean(), gbscores13.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores14.min(), gbscores14.mean(), gbscores14.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores15.min(), gbscores15.mean(), gbscores15.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores16.min(), gbscores16.mean(), gbscores16.max()))

Gradient Boosted Trees CV scores:
min: 0.557, mean: 0.639, max: 0.712
min: 0.552, mean: 0.642, max: 0.710
min: 0.552, mean: 0.642, max: 0.710
min: 0.552, mean: 0.642, max: 0.710


For now I'll stick with Gradient Boosting as my classifier and I'll start munging the test data that I'll be using to make my predictions with. First I need to get the sample submission csv that Kaggle gave us into a format that I can use with scikit-learn.

In [41]:
ss['year'] = ss.id.str.split('_').str.get(0)
ss['team1'] = ss.id.str.split('_').str.get(1)
ss['team2'] = ss.id.str.split('_').str.get(2)
ss.drop('pred', axis=1, inplace=True)
ss[['year', 'team1', 'team2']] = ss[['year', 'team1', 'team2']].apply(pd.to_numeric)
ss13 = ss[ss['year']==2013]
ss14 = ss[ss['year']==2014]
ss15 = ss[ss['year']==2015]
ss16 = ss[ss['year']==2016]


In [42]:
rs2013_end = pd.DataFrame()
rs2014_end = pd.DataFrame()
rs2015_end = pd.DataFrame()
rs2016_end = pd.DataFrame()

In [43]:
rs2013_end = rs2013_train.groupby('team1').last().reset_index(drop=False)
rs2014_end = rs2014_train.groupby('team1').last().reset_index(drop=False)
rs2015_end = rs2015_train.groupby('team1').last().reset_index(drop=False)
rs2016_end = rs2016_train.groupby('team1').last().reset_index(drop=False)

In [44]:
t1_2013_end = rs2013_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss13 = pd.merge(ss13, t1_2013_end, on='team1')
t2_2013_end = pd.DataFrame()
t2_2013_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2013_end
ss13 = pd.merge(ss13, t2_2013_end, on='team2')

In [45]:
t1_2014_end = rs2014_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss14 = pd.merge(ss14, t1_2014_end, on='team1')
t2_2014_end = pd.DataFrame()
t2_2014_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2014_end
ss14 = pd.merge(ss14, t2_2014_end, on='team2')

In [46]:
t1_2015_end = rs2015_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss15 = pd.merge(ss15, t1_2015_end, on='team1')
t2_2015_end = pd.DataFrame()
t2_2015_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2015_end
ss15 = pd.merge(ss15, t2_2015_end, on='team2')

In [47]:
t1_2016_end = rs2016_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss16 = pd.merge(ss16, t1_2016_end, on='team1')
t2_2016_end = pd.DataFrame()
t2_2016_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2016_end
ss16 = pd.merge(ss16, t2_2016_end, on='team2')

In [48]:
test_13 = ss13[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_14 = ss14[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_15 = ss15[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_16 = ss16[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values

In [49]:
gb.fit(features_array13, target13)
t1_pred_prob_2013 = gb.predict_proba(test_13)

gb.fit(features_array14, target14)
t1_pred_prob_2014 = gb.predict_proba(test_14)

gb.fit(features_array15, target15)
t1_pred_prob_2015 = gb.predict_proba(test_15)

gb.fit(features_array16, target16)
t1_pred_prob_2016 = gb.predict_proba(test_16)

In [50]:
preds13 = t1_pred_prob_2013[:,1]
ss13['pred'] = preds13
preds14 = t1_pred_prob_2014[:,1]
ss14['pred'] = preds14
preds15 = t1_pred_prob_2015[:,1]
ss15['pred'] = preds15
preds16 = t1_pred_prob_2016[:,1]
ss16['pred'] = preds16

In [51]:
ss13 = ss13[['id', 'pred']]
ss14 = ss14[['id', 'pred']]
ss15 = ss15[['id', 'pred']]
ss16 = ss16[['id', 'pred']]

In [52]:
ss13 = ss13.sort_values('id').reset_index(drop=True)
ss14 = ss14.sort_values('id').reset_index(drop=True)
ss15 = ss15.sort_values('id').reset_index(drop=True)
ss16 = ss16.sort_values('id').reset_index(drop=True)

In [63]:

ss_3_10_1635 = pd.concat([ss13, ss14, ss15, ss16], ignore_index=True)

In [67]:
ss_3_10_1635.to_csv('ss_3_10_1635.csv', index=False)